In [ ]:
import numpy as np
import pandas as pd
from datasets import concatenate_datasets, load_dataset

Reading the raw dataset.

In [ ]:
ds = load_dataset(
    "stanfordnlp/imdb",
    revision="e6281661ce1c48d982bc483cf8a173c1bbeb5d31"
)
df = concatenate_datasets(ds.values()).to_pandas()
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["label"])

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body"})

Stripping the bodies.

In [ ]:
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["non-political"] * len(df))

Inspecting the body length.

In [ ]:
df["body_length"] = df["body"].fillna("").str.len()
df["body_word_count"] = df["body"].fillna("").str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
df["body_length"].mean()

In [ ]:
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df.to_parquet("../preprocessed/imdb.parquet")